Document for Model training



Importing packages

In [1]:
import random
import math
import matplotlib.pyplot as plt
from PIL import ImageDraw, Image
import bifurcationMathFunctions as bmf
import numpy as np

Setting Constants

In [3]:
startX = 5
startY = 200
startAngle = 0
starWidth = 15
stopWidth = 2
startLength = 20
bifurcProb = 0.3
bifurcBigLeft = 0.5

Synthetic Angiogram Class: Node

In [4]:
class Node:
    def __init__(self, coord, width, length, angle):
        self.coord = coord
        self.width = width
        self.length = length
        self.angle = angle
        self.children = []

    #Creates a child node
    def createChild(self, angle, ratio):
        newWidth = self.width*ratio
        newLength = self.length*ratio
        curX = self.coord[0]
        curY = self.coord[1]
        newX = curX + math.cos(angle)*newLength
        newY = curY + math.sin(angle)*newLength
        newCoord = newX, newY
        return Node(newCoord, newWidth, newLength, angle)

    #Creates and add either 1 or 2 children to the parent node.
    def addChildren(self):
        bifurcation = (random.random() < bifurcProb)
        if bifurcation:
            # alpha is the ratio between diameters small/big
            alpha = bmf.getRandomAlpha()
            # Make 2 children mirrored across an angle
            # getAllParameters returns (bigParams, smallParams).
            # The bifurcBigLeft randomises direction of biggest new artery
            if bifurcBigLeft < random.random():
                (ratioL, ratioR), (angleL, angleR) = bmf.getAllParameters(alpha)
            else:
                (ratioR, ratioL), (angleR, angleL) = bmf.getAllParameters(alpha)
            leftChild = self.createChild(self.angle-angleL, ratioL)
            self.children.append(leftChild)
            rightChild = self.createChild(self.angle+angleR, ratioR)
            self.children.append(rightChild)
        else:
            #Make 1 child following an angle
            singleChild = self.createChild(self.angle, 1)
            self.children.append(singleChild)

Synthetic Angiogram Class: Tree

In [5]:
class Tree:
    def __init__(self, x, y, width, length, angle):
        self.addRoot(x,y, width, length, angle)
        self.makeTree()

    def addRoot(self, x, y, width, length, angle):
        self.root = Node((x,y), width, length, angle)
    
    def growTree(self, node):
        if node.width < stopWidth:
            return
        else:
            node.addChildren()
            for child in node.children:
                self.growTree(child)
    
    def makeTree(self):
        self.growTree(self.root)
        

Drawing Tree

In [6]:
def nodeInside(cx, cy, X, Y):
    return (cx < X and cx >= 0 and cy < Y and cy >= 0)

def drawNode(node, draw):
    if len(node.children) >= 1:
        px, py = int(node.coord[0]), int(node.coord[1])
        for child in node.children:
            cx, cy = int(child.coord[0]), int(child.coord[1])
            draw.line((px, py, cx, cy), fill = 1, width=int(child.width))
            drawNode(child, draw)
    else:
        return

#Makes a treewalk, and finds minimum and maximum x, y values for tree
def drawTree(tree, dim):
    img = Image.new("1", dim)
    draw = ImageDraw.Draw(img)
    drawNode(tree.root, draw)
    img = np.array(img)
    #plt.imshow(img,cmap="gray")
    return img
#help function that draws line from parent to each of its children. Should check for borders

Executing

In [8]:
tree = Tree(startX, startY, starWidth, startLength, startAngle)
synAngio = drawTree(tree, (736, 736))

for i in range(10):
    tree = Tree(startX, startY, starWidth, startLength, startAngle)
    synAngio = drawTree(tree, (736, 736))

In [54]:
#synAngio.save("test_image.PNG")